In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
from glob import glob

import numpy as np
import xarray as xr
import pandas as pd

import config
import util

In [5]:
cluster, client = util.get_ClusterClient()
cluster.scale(12)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.60:33304,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [15]:
USER = os.environ['USER']
dirout = f'/glade/work/{USER}/sno-analysis/sno-flux-products'

flux_files = sorted(glob(f'{dirout}/*.nc'))

convert = dict(
    CO2={'factor': 12.01e-15 * 86400.0 * 365.25, 'units': 'PgC/yr'},
    O2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
    APO={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},    
    N2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
)

index = []
columns = []
data = []
for n, f in enumerate(flux_files):
    v = os.path.basename(f).split('.')[0]
    product = os.path.basename(f).split('.')[1]
    print(f'{v}: {product}')
          
    constituent = [c for c in ['CO2', 'O2', 'N2', 'APO'] if c in v][0]
    scale_by = convert[constituent]['factor']
    units = convert[constituent]['units']
    columns.append(f'{v} ({product}) [{units}]')
    
    with xr.open_dataset(f, chunks={'time': 100}) as ds:
        total = (ds[v] * ds.area).sum(['lat', 'lon'])
        total *= scale_by
        data.append(total.values)
        if n == 0:
            index = total.time.values
        else:
            assert (index == total.time.values).all()

df = pd.DataFrame(data=np.array(data).T, index=index, columns=columns)
df            

SFAPO_OCN: carboscope
SFCO2_FF: GCP-GridFED
SFCO2_FF: OCO2-MIP
SFCO2_OCN: MPI-SOM-FFN
SFCO2_OCN: cesm_fosi_smyle
SFN2_OCN: cesm_fosi_smyle
SFN2_OCN: era5_shf
SFO2_FF: GCP-GridFED
SFO2_OCN: cesm_fosi_smyle
SFO2_OCN: gk2001_R2016


,SFAPO_OCN (carboscope) [Pmol/yr],SFCO2_FF (GCP-GridFED) [PgC/yr],SFCO2_FF (OCO2-MIP) [PgC/yr],SFCO2_OCN (MPI-SOM-FFN) [PgC/yr],SFCO2_OCN (cesm_fosi_smyle) [PgC/yr],SFN2_OCN (cesm_fosi_smyle) [Pmol/yr],SFN2_OCN (era5_shf) [Pmol/yr],SFO2_FF (GCP-GridFED) [Pmol/yr],SFO2_OCN (cesm_fosi_smyle) [Pmol/yr],SFO2_OCN (gk2001_R2016) [Pmol/yr]
1986-01-01 12:00:00,0.000000,6.349926,0.000000,-1.566164,-2.629393,0.033259,0.476587,-0.733670,0.144992,0.653724
1986-01-02 12:00:00,0.000000,6.358007,0.000000,-1.561373,-2.567423,0.029607,0.473220,-0.734536,0.231176,0.653285
1986-01-03 12:00:00,0.000000,6.366089,0.000000,-1.556582,-2.464227,0.025954,0.469853,-0.735402,0.382042,0.652845
1986-01-04 12:00:00,0.000000,6.374168,0.000000,-1.551790,-2.945662,0.022302,0.466486,-0.736268,0.056199,0.652405
1986-01-05 12:00:00,0.000000,6.382249,0.000000,-1.546999,-2.911721,0.018650,0.463120,-0.737134,-0.011542,0.651965
...,...,...,...,...,...,...,...,...,...,...
2020-12-27 12:00:00,1.423361,10.787069,10.116341,-3.187002,-4.581169,0.076151,0.408909,-1.241463,0.233372,0.655923
2020-12-28 12:00:00,1.457859,10.808172,10.806556,-3.197571,-4.129595,0.076168,0.410834,-1.244030,0.391277,0.655483
2020-12-29 12:00:00,1.489718,10.829274,10.856695,-3.208139,-4.241344,0.076185,0.412759,-1.246597,0.406275,0.655044
2020-12-30 12:00:00,1.518499,10.850378,10.855813,-3.218708,-5.244919,0.076202,0.414685,-1.249164,0.237832,0.654604


In [16]:
df.to_csv(f'{dirout}/daily-totals.csv')